# Set-up

In [1]:
# Import modules
import os
import torch

# ML4G
import seqdata as sd

# EUGENe
from eugene.models import zoo
from eugene import models
from eugene import train

In [2]:
# Change working dir
os.chdir("/cellar/users/aklie/data/datasets/deBoer_random-promoters_GPRA/")

In [3]:
# Set log dir
date = "2023_12_24"
dataset = "dream"
model = "Basset"
target_var = "bin"
train_var = "train_val"

log_dir = f"models/{date}/{dataset}/{model}/{target_var}/{train_var}"
print(f"Log dir: {log_dir}")

Log dir: models/2023_12_24/dream/Basset/bin/train_val


In [4]:
# cuda check
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

cuda:0


# Load data

In [5]:
sdata = sd.open_zarr("training/2023_12_24/dream/training.zarr")
sdata

<xarray.Dataset>
Dimensions:       (_sequence: 6739258, length: 150, _ohe: 4, _probs: 18)
Coordinates:
    index         (_sequence) int64 dask.array<chunksize=(105301,), meta=np.ndarray>
Dimensions without coordinates: _sequence, length, _ohe, _probs
Data variables:
    bin           (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    fold          (_sequence) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    is_singleton  (_sequence) bool dask.array<chunksize=(1000,), meta=np.ndarray>
    ohe_seq       (_sequence, length, _ohe) uint8 dask.array<chunksize=(1000, 150, 4), meta=np.ndarray>
    probs_bin     (_sequence, _probs) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    seq           (_sequence) <U150 dask.array<chunksize=(1000,), meta=np.ndarray>
    train_val     (_sequence) bool dask.array<chunksize=(1000,), meta=np.ndarray>

In [6]:
test_seqs = torch.tensor(sdata["ohe_seq"][:10].values.transpose(0, 2, 1), dtype=torch.float32)
test_seqs.shape

torch.Size([10, 4, 150])

# Load model

In [7]:
# Instantiate architecture
arch = zoo.Basset(
    input_len=150,
    output_dim=1
)
arch(test_seqs).shape

torch.Size([10, 1])

In [8]:
# Instantiate a sequence module
module = models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer_lr=0.002,
    optimizer_kwargs=dict(weight_decay=1e-6),
    scheduler="reduce_lr_on_plateau",
    scheduler_monitor="val_loss_epoch",
    metric="pearson",
    seed=1234,
)
module

[rank: 0] Global seed set to 1234


SequenceModule(
  (arch): Basset(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 300, kernel_size=(19,), stride=(1,), padding=(9,))
        (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
        (4): Conv1d(300, 200, kernel_size=(11,), stride=(1,), padding=(5,))
        (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
        (8): Conv1d(200, 200, kernel_size=(7,), stride=(1,), padding=(3,))
        (9): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (dense_block): DenseBlock(
      (layers): Sequential(

# Training

In [9]:
def seq_transform(x):
    return x.swapaxes(1, 2)

In [10]:
# Train the model
trainer = train.fit_sequence_module(
    model=module,
    sdata=sdata,
    seq_var="ohe_seq",
    target_vars="bin",
    in_memory=True,
    train_var="train_val",
    transforms={"ohe_seq": seq_transform},
    epochs=100,
    batch_size=1024,
    early_stopping_monitor=f"val_loss_epoch",
    early_stopping_patience=10,
    early_stopping_mode="min",
    model_checkpoint_monitor=f"val_loss_epoch",
    model_checkpoint_k=5,
    model_checkpoint_mode="min",
    gpus="auto",
    logger="csv",
    log_dir=log_dir,
    name="",
    version="",
    seed=1234,
    return_trainer=True
)

Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['bin'] into memory


[rank: 0] Global seed set to 1234
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
LOCAL_

Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# Move best model to log directory
best_model_path = trainer.checkpoint_callback.best_model_path
copy_path = os.path.join(log_dir, "best_model.ckpt")
os.system(f"cp {best_model_path} {copy_path}")

# DONE!

---